<a href="https://colab.research.google.com/github/pg-sys/astra_chatbot_documentQA/blob/main/astra_vsearch_QA_for_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# astra-vsearch-QA-for-documents
This demo guides you through setting up Astra DB with Vector Search, Cassio and Open AI to implement an generative Q&A for your own Documentation

Jupyter notebook for generative Q&A for douments is powered by [Astra Vector Search](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html) and [OpenAI](https://github.com/openai/) and Casssio [Opensource LLM integration with Cassandra and Astra DB](https://cassio.org/).

## Astra Vector Search
Astra vector search enables developers to search a database by context or meaning rather than keywords or literal values. This is done by using “embeddings”. Embeddings are a type of representation used in machine learning where high-dimensional or complex data is mapped onto vectors in a lower-dimensional space. These vectors capture the semantic properties of the input data, meaning that similar data points have similar embeddings.
Reference: [Astra Vector Search](https://docs.datastax.com/en/astra-serverless/docs/vector-search/overview.html)

## CassIO
CassIO is the ultimate solution for seamlessly integrating Apache Cassandra® with generative artificial intelligence and other machine learning workloads. This powerful Python library simplifies the complicated process of accessing the advanced features of the Cassandra database, including vector search capabilities. With CassIO, developers can fully concentrate on designing and perfecting their AI systems without any concerns regarding the complexities of integration with Cassandra.
Reference [Cassio](https://cassio.org/)

## OpenAI
OpenAI provides various tools and resources to implement your own Document QA Search system. This includes pre-trained language models like GPT-3.5, which can understand and generate human-like text. Additionally, OpenAI offers guidelines and APIs to leverage their models for document search and question-answering tasks, enabling developers to build powerful and intelligent Document QA Search applications.
Reference: [OpenAI](https://github.com/openai/)

## Demo Summary
ChatGPT excels at answering questions, but only on topics it remembers from its training data. It offers you a nice dialog interface to ask questions and get answers.

But what do you do when you have your onw documents? How can you leverage the GenAI and LLM models to get insights in those?

Think of an Q/A Bot that you want to provide to your customers for asking questions against the documentation of your products.

For beeing able to do so, you have to implement your own ChatGPT-like solution.
The implementation requires
1. Analysing your existing documents and store the information
2. Providing search capabilities for your questions to get answers

This is solve by using a LLM models. Ideally you embedd the data as vectors and store them in a vector database and then use the LLM models on top of that database.

This notebook demonstrates a two-step Search-Ask method for enabling GPT to answer questions using a library of reference on your onw documentations based on Astra DB vector search.




# Getting Started with this notebook

These are prerequisites you need to to before running this notebook
- Create a new vector search enabled database in Astra.
- Create a keyspace
- Create a token with permissions to create tables
- Download your secure-connect-bundle.zip file
- Create an OpenAI account and download an API Key

- When you run this notebook, it will ask you for providing the secure-connect-bundle.zip, some text file and client ids, passwords as well as API Key

# Setup

This jupyter notebook was build on Colab. You need to install the following libraries.

In [23]:
# install required dependencies
! pip install \
    "cassandra-driver>=3.28.0" \
    "openai==0.27.7" \
    "tiktoken==0.4.0" \
    "langchain>=0.0.218" \
    "cassio==0.0.7" \
    "pypdf"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.5/254.5 kB 5.7 MB/s eta 0:00:00


# Imports

In [ ]:
# Imports for our environment and accessing Astra DB
import os

import getpass
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from google.colab import files

# Astra DB configuration, connection bundle and token secrets

You will need a secure connect bundle and a user with access permission. For demo purposes the "administrator" role will work fine.
More information about how to get the bundle can be found [here](https://docs.datastax.com/en/astra-serverless/docs/connect/secure-connect-bundle.html).

In [60]:
#upload secure connect bundle
print('Please upload your Secure Connect Bundle')
uploaded = files.upload()
if uploaded:
    astraBundleFileTitle = list(uploaded.keys())[0]
    SECURE_CONNECT_BUNDLE_PATH = os.path.join(os.getcwd(), astraBundleFileTitle)
else:
    raise ValueError(
        'Cannot proceed without Secure Connect Bundle. Please re-run the cell.'
    )
#Alternatively upload to the environment and reference it here
#SECURE_CONNECT_BUNDLE_PATH = '/content/secure-connect-documentation.zip'


Please upload your Secure Connect Bundle


Saving secure-connect-documentation.zip to secure-connect-documentation (1).zip


In [ ]:
ASTRA_DB_TOKEN_BASED_USERNAME = getpass.getpass('What Astra DB token username do you want to use? ')
#ASTRA_DB_TOKEN_BASED_USERNAME = '<<ENTER>>'

In [ ]:
ASTRA_DB_TOKEN_BASED_PASSWORD = getpass.getpass('What Astra DB token password do you want to use? ')
#ASTRA_DB_TOKEN_BASED_PASSWORD = '<<ENTER>>'

In [ ]:
ASTRA_DB_KEYSPACE = input(f'Which Astra DB keypsace do you want to use? ')
#ASTRA_DB_KEYSPACE = 'mykeyspace'

# Provide Sample Data
If you want to provide some docoments, you can upload them here.
As a sample document you can also download some text here:

In [69]:
# retrieve the text of a short story that will be indexed in the vector store
! curl https://raw.githubusercontent.com/CassioML/cassio-website/main/docs/frameworks/langchain/texts/amontillado.txt --output amontillado.txt
SAMPLEDATA = ["amontillado.txt"]

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13022  100 13022    0     0  77511      0 --:--:-- --:--:-- --:--:-- 77053


In [61]:
# Alternatively you can provide your own file - please consider to customize the queries at the end of the notebook to match your content.
#provide some sample files
print('Please upload your own sample file:')
uploaded = files.upload()
if uploaded:
    #sampleDataFileTitle = list(uploaded.keys()[0])
    #SAMPLEDATA_PATH = os.path.join(os.getcwd(), sampleDataFileTitle)
    SAMPLEDATA = uploaded
else:
    raise ValueError(
        'Cannot proceed without Sample Data. Please re-run the cell.'
    )

Please upload your own sample file:


Saving Customer_Case_Study_Netflix.pdf to Customer_Case_Study_Netflix.pdf
Saving Customer_Case_Study_eBay.pdf to Customer_Case_Study_eBay.pdf


# Connect to Astra DB

In [62]:
# make sure that you can connect to Astra DB - if you see errors, then have a look at the environment you configured earlier

cloud_config = {
   'secure_connect_bundle': SECURE_CONNECT_BUNDLE_PATH
}
auth_provider = PlainTextAuthProvider(ASTRA_DB_TOKEN_BASED_USERNAME, ASTRA_DB_TOKEN_BASED_PASSWORD)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(140334051140528) cfe0851c-36a8-420c-9b15-ac1200c177c7-us-east1.db.astra.datastax.com:29042:923a1564-a24c-4d30-a97c-ef47bce2ad18> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


# Read files, Create Embeddings, Store in Vector DB

CassIO seamlessly integrates with LangChain, offering Cassandra-specific tools for many tasks. In our example we will use vector stores, indexers, embeddings and queries.

And we will use OpenAI for our LLM services. (See Pre-requisites on [cassio.org](https://cassio.org/start_here/#llm-access) for more details).

In [63]:
# Set your secret(s) for LLM access:
# we will use OpenAI embeddings, so please provide your OpenAI AKP Key
apiSecret = getpass.getpass('Your secret for LLM provider OpenAI: ')
#apiSecret = "<<ENTER>>"
os.environ['OPENAI_API_KEY'] = apiSecret

Your secret for LLM provider OpenAI: ··········


In [70]:
#Import the needed libraries and declare the LLM model
import langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Cassandra
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
import os

# define the embedding function to use
embedding_function = OpenAIEmbeddings()

#define the table name to be used to store our embeddings, Cassio will create the objects in Astra DB for you.
ASTRA_DB_TABLE_NAME = 'vdocuments'


# now loop through all files uploaded and process them
for elem in SAMPLEDATA:
  filename = elem
  doc_path = os.path.join(os.getcwd(), filename)

  # check which filetype they are and parse them (loader)
  # check if the file is a PDF
  if filename.endswith(".pdf"):
		# load the PDF file
    loader = PyPDFLoader(doc_path)
    pages = loader.load_and_split()
    print(f"Processed PDF file: {filename}")

  # check if the file is a TXT
  elif filename.endswith(".txt"):
    loader = TextLoader(doc_path)
    pages = loader.load_and_split()
    print(f"Processed TXT file: {filename}")

  # other files will not be processed
  else:
    # handle the case where the file has an unsupported extension
    print(f"Unsupported file type: {filename}")


  # if any file was loaded, proceed
  if len(pages) >0:

    #create a vector store object for the document, that will automatically embedd it
    cassVStore = Cassandra.from_documents(
      documents=pages,
      embedding=embedding_function,
      session=session,
      keyspace=ASTRA_DB_KEYSPACE,
      table_name=ASTRA_DB_TABLE_NAME,
    )

    # now clean up
    # delete the file
    os.remove(doc_path)
    # empty pages
    pages = ""

# empty the list of file names, just in case this block is run twice.
SAMPLEDATA = []

print(f"\nProcessing done.")

Processed TXT file: amontillado.txt


100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


Processing done.


In [59]:
# just in case this demo runs multiple times and you want to clean up, run this:
#cassVStore.clear()

# Now Query the Data and execute some "searches" against it
First we will start with a similarity search using the Vectorstore's implementation

In [71]:
# similarity search:
prompt = "What die Luchesi say about Nitro?"

# matched_docs is a list with the found documents from the similarity search
matched_docs = cassVStore.similarity_search(prompt)
# for each of the found documents, print the content
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

I took from their sconces two flambeaux, and giving one to Fortunato,
bowed him through several suites of rooms to the archway that led into
the vaults.  I passed down a long and winding staircase, requesting him
to be cautious as he followed. We came at length to the foot of the
descent, and stood together on the damp ground of the catacombs of the
Montresors.

The gait of my friend was unsteady, and the bells upon his cap jingled
as he strode.

"The pipe," said he.

"It is farther on," said I; "but observe the white web-work which
gleams from these cavern walls."

He turned towards me, and looked into my eyes with two filmy orbs that
distilled the rheum of intoxication.

"Nitre?" he asked, at length.

"Nitre," I replied.  "How long have you had that cough?"

"Ugh! ugh! ugh!--ugh! ugh! ugh!--ugh! ugh! ugh!--ugh! ugh! ugh!--ugh!
ugh! ugh!"

My poor friend found it impossible to reply for many minutes.

"It is nothing," he said, at last.

"Come," I said, with decision, "

# Finally do a Q/A Search
To be able implement question answering over documents we need to do four steps:

1. Create an index on top of our Vector store
2. Create a Retriever from that index
3. Ask questions (promts)!


A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) it. Vector stores can be used as the backbone of a retriever.

Hint: The query method is creating a chain using OpenAI document Search:

        llm = llm or OpenAI(temperature=0)
        chain = RetrievalQA.from_chain_type(
            llm, retriever=self.vectorstore.as_retriever(), **kwargs
        )

In [72]:
# Q/A LLM Search
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
index = VectorStoreIndexWrapper(vectorstore=cassVStore)

# Search within the document context for some text related information.
prompt = "Who is Luchesi?"
index.query(prompt)


' Luchesi is a character mentioned in the story. He is a connoisseur of wine who Fortunato believes can tell Amontillado from Sherry.'

In [73]:
#Alternatively you can use a retrieval chain and some conversation history
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
#vectordb.persist()
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
pdf_qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.8) , cassVStore.as_retriever(), memory=memory)


query = "Who is Luchesi?"
result = pdf_qa({"question": query})
print("Answer:")
result["answer"]

Answer:


' Luchesi is an Italian connoisseur of wines who Fortunato suggests Montresor should consult to compare the Amontillado with Sherry.'

# Other Usefull stuff:  
E.g. query the Vector Store to see what has been added to it and what happended with our documentation

In [74]:
cqlSelect = f'SELECT * FROM {ASTRA_DB_KEYSPACE}.{ASTRA_DB_TABLE_NAME} LIMIT 3;'  # (Not a production-optimized query ...)
rows = session.execute(cqlSelect)
for row_i, row in enumerate(rows):
    print(f'\nRow {row_i}:')
    print(f'    document_id:      {row.document_id}')
    print(f'    embedding_vector: {str(row.embedding_vector)[:64]} ...')
    print(f'    document:         {row.document[:64]} ...')
    print(f'    metadata_blob:    {row.metadata_blob}')

print('\n...')


Row 0:
    document_id:      4d6919f113224f0081bc794e68185d19
    embedding_vector: [-0.004671068396419287, -0.0024927675258368254, 0.01194973755627 ...
    document:         At the most remote end of the crypt there appeared another less
 ...
    metadata_blob:    {"source": "/content/amontillado.txt"}

Row 1:
    document_id:      ba738dd0a394409d90c3ef29b0db13a1
    embedding_vector: [-0.0056596905924379826, -0.01846846379339695, -0.01377687882632 ...
    document:         DataStax Headquarters
San Francisco Bay Area
3975 Freedom Circle ...
    metadata_blob:    {"source": "/content/Customer_Case_Study_Netflix.pdf", "page": 1}

Row 2:
    document_id:      c144a9c2577c4f808c80b5bc61f884a2
    embedding_vector: [-0.012555774301290512, -0.004729681648313999, -0.00012366501323 ...
    document:         PERSONALIZATION
EBAY ENGAGES CUSTOMERS WITH  
PERSONALIZED RECOM ...
    metadata_blob:    {"source": "/content/Customer_Case_Study_eBay.pdf", "page": 0}

...
